# Notebook 5.6: (Optional Advanced Mini-Project) Integrated Fed-Batch Bioreactor Optimization with NMPC/EMPC and State Estimation

This notebook serves as an advanced mini-project, bringing together several key concepts we've explored in Part 5: nonlinear bioreactor modeling, NMPC/EMPC design with CasADi, and state estimation using an Extended Kalman Filter (EKF).

Our goal is to simulate a more realistic scenario where an NMPC or EMPC controller optimizes the fed-batch bioreactor operation based on **estimated states** derived from noisy measurements, rather than assuming perfect state knowledge.

**Project Objectives:**
1.  Set up a simulated fed-batch bioreactor "plant" that includes process and measurement noise.
2.  Implement (or re-use) an Extended Kalman Filter (EKF) for estimating the bioreactor states ($X_v, S, P, V$).
3.  Implement (or re-use) either the glucose setpoint-tracking NMPC (from Notebook 5.2) or the product-maximizing EMPC (from Notebook 5.3).
4.  Run a closed-loop simulation integrating the plant, EKF, and NMPC/EMPC.
5.  Analyze the overall system performance: achievement of control/economic objectives, quality of state estimation, and constraint handling.
6.  (Optional) Briefly explore the impact of EKF tuning or MPC tuning on the integrated system.

## 1. Importing Libraries and Re-using Core Functions

We will reuse the bioreactor model, EKF class, and NMPC/EMPC CasADi setup functions from previous notebooks (5.1, 5.2, 5.3, 5.4). Ensure these are either copied here or accessible (e.g., if this notebook is in the same directory or helper functions are in a `.py` file).

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import solve_ivp
import casadi as ca

# Optional: for nicer plots
plt.rcParams.update({'font.size': 12, 'figure.figsize': (14, 16)}) 

# --- Assume functions from previous notebooks are defined here or imported ---
# bioreactor_ode_numpy (from 5.1)
# bioreactor_ode_casadi (from 5.1/5.2)
# ExtendedKalmanFilter class (from 5.4, including f_discrete_func, Ak_jacobian_func, etc.)
# NMPC/EMPC CasADi setup functions (from 5.2 or 5.3)

# For brevity, let's redefine the essential ones here. 
# In a real project, these would be in well-organized helper files.

default_params = {
    'mu_max': 0.08, 'K_S': 0.1, 'k_d': 0.005, 'k_d_S': 0.02, 'K_d_S_coeff': 0.01,
    'Y_XS': 0.5, 'm_S': 0.01, 'alpha': 0.01, 'beta': 0.002,
    'Y_PS': 1e9, 'S_feed': 200.0
}
nx = 4; nu = 1
Ts_control = 1.0 # Control and EKF interval

def bioreactor_ode_numpy_plant(t, states, params, F_in_val):
    # This is the 'true' plant model for simulation
    Xv, S, P, V = states
    # (kinetics as in 5.1)
    mu_max, K_S, k_d, k_d_S, K_d_S_coeff = params['mu_max'], params['K_S'], params['k_d'], params['k_d_S'], params['K_d_S_coeff']
    Y_XS, m_S, alpha, beta, Y_PS, S_feed = params['Y_XS'], params['m_S'], params['alpha'], params['beta'], params['Y_PS'], params['S_feed']
    mu = mu_max * S / (K_S + S + 1e-9)
    mu_d_val = k_d + k_d_S * K_d_S_coeff / (K_d_S_coeff + S + 1e-9)
    q_P = alpha * mu + beta
    if Y_PS > 1e6: q_S = (mu / (Y_XS + 1e-9)) + m_S
    else: q_S = (mu / (Y_XS + 1e-9)) + m_S + (q_P / (Y_PS + 1e-9))
    if V < 1e-6: return [0,0,0,F_in_val]
    dilution = F_in_val / V
    dXv_dt = (mu - mu_d_val) * Xv - dilution * Xv
    dS_dt = -q_S * Xv + dilution * (S_feed - S)
    dP_dt = q_P * Xv - dilution * P
    dV_dt = F_in_val
    return [dXv_dt, dS_dt, dP_dt, dV_dt]

# --- EKF Class and CasADi functions for EKF (condensed from 5.4) ---
x_s_ekf = ca.SX.sym('x_s_ekf', nx); u_s_ekf = ca.SX.sym('u_s_ekf', nu)
ode_rhs_ekf_model = bioreactor_ode_casadi(x_s_ekf, default_params, u_s_ekf) # Model used by EKF
dae_ekf_model = {'x': x_s_ekf, 'p': u_s_ekf, 'ode': ode_rhs_ekf_model}
intg_opts_ekf_model = {'tf': Ts_control, 'simplify': True, 'number_of_finite_elements': 4}
F_discrete_integrator_ekf = ca.integrator('F_ekf', 'rk', dae_ekf_model, intg_opts_ekf_model)
x_k_plus_1_ekf_sym = F_discrete_integrator_ekf(x0=x_s_ekf, p=u_s_ekf)['xf']
f_discrete_ekf_func = ca.Function('f_ekf', [x_s_ekf, u_s_ekf], [x_k_plus_1_ekf_sym])
Ak_jacobian_ekf_func = ca.Function('Ak_ekf', [x_s_ekf, u_s_ekf], [ca.jacobian(x_k_plus_1_ekf_sym, x_s_ekf)])
num_measured_outputs_ekf = 2 # Assume S and V are measured
h_measurement_ekf_sym = ca.vertcat(x_s_ekf[1], x_s_ekf[3]) # S, V
h_measurement_ekf_func = ca.Function('h_ekf', [x_s_ekf, u_s_ekf], [h_measurement_ekf_sym])
Ck_jacobian_ekf_func = ca.Function('Ck_ekf', [x_s_ekf, u_s_ekf], [ca.jacobian(h_measurement_ekf_sym, x_s_ekf)])

class ExtendedKalmanFilterProject(ExtendedKalmanFilter): # Inherit from 5.4 if it's defined
    # Or re-paste the EKF class definition from Notebook 5.4 here
    def __init__(self, f_disc_func, h_meas_func, Ak_jac_func, Ck_jac_func, 
                 Qk_mat, Rk_mat, x_hat0, P0, n_measure_out):
        self.f = f_disc_func; self.h = h_meas_func; self.Ak_jac = Ak_jac_func; self.Ck_jac = Ck_jac_func
        self.Qk = Qk_mat; self.Rk = Rk_mat
        self.x_hat = np.array(x_hat0).reshape(-1,1); self.P = np.array(P0)
        self.nx = self.x_hat.shape[0]; self.ny_meas = n_measure_out
    def predict(self, uk_minus_1):
        uk_prev_num = np.array(uk_minus_1).reshape(-1,1)
        Ak_val = np.array(self.Ak_jac(self.x_hat, uk_prev_num))
        self.x_hat_apriori = np.array(self.f(self.x_hat, uk_prev_num)).reshape(-1,1)
        self.P_apriori = Ak_val @ self.P @ Ak_val.T + self.Qk
    def update(self, y_measured_k, uk_current):
        uk_curr_num = np.array(uk_current).reshape(-1,1); y_m_num = np.array(y_measured_k).reshape(-1,1)
        Ck_val = np.array(self.Ck_jac(self.x_hat_apriori, uk_curr_num))
        y_pred_h = np.array(self.h(self.x_hat_apriori, uk_curr_num)).reshape(-1,1)
        innovation = y_m_num - y_pred_h
        S_innovation = Ck_val @ self.P_apriori @ Ck_val.T + self.Rk
        Kk = self.P_apriori @ Ck_val.T @ np.linalg.inv(S_innovation)
        self.x_hat = self.x_hat_apriori + Kk @ innovation
        I_nx = np.eye(self.nx)
        self.P = (I_nx - Kk @ Ck_val) @ self.P_apriori
        self.P = 0.5 * (self.P + self.P.T)
    def step(self, uk_prev_for_predict, y_measured_k, uk_curr_for_update):
        self.predict(uk_prev_for_predict)
        self.update(y_measured_k, uk_curr_for_update)
        return self.x_hat.copy(), self.P.copy()

## 2. Project Setup: Plant with Noise, EKF, and NMPC/EMPC Choice

### 2.1 Plant Simulation Parameters (with Noise)

In [ ]:
# Plant initial conditions
x_true_initial = np.array([0.1, 5.0, 0.0, 1.0]) # [Xv0, S0, P0, V0]

# Noise characteristics for the 'true' plant simulation
sigma_S_meas_plant = 0.15   # Std dev of Glucose measurement noise for plant
sigma_V_meas_plant = 0.015  # Std dev of Volume measurement noise for plant
Rk_plant_true = np.diag([sigma_S_meas_plant**2, sigma_V_meas_plant**2])

sigma_Xv_proc_plant = 0.02 
sigma_S_proc_plant  = 0.1  
sigma_P_proc_plant  = 0.002
sigma_V_proc_plant  = 0.002
Qk_plant_true = np.diag([sigma_Xv_proc_plant**2, sigma_S_proc_plant**2, 
                           sigma_P_proc_plant**2, sigma_V_proc_plant**2])

print("Plant noise parameters defined.")

### 2.2 EKF Configuration

We need to provide the EKF with its *assumed* $Q_K$ and $R_K$. These might differ from the true plant noise characteristics, reflecting model mismatch for the estimator.

In [ ]:
# EKF's assumed noise covariances (can be tuned)
Rk_ekf_assumed = np.diag([0.1**2, 0.01**2]) # EKF's belief about measurement noise
Qk_ekf_assumed = np.diag([0.01**2, 0.05**2, 0.001**2, 0.001**2]) # EKF's belief about process noise

x_hat0_ekf_project = np.array([0.12, 4.8, 0.005, 1.005]).reshape(nx,1) # Initial estimate
P0_ekf_project = np.diag([0.1**2, 0.5**2, 0.01**2, 0.02**2])    # Initial estimate covariance

ekf_instance = ExtendedKalmanFilterProject(f_discrete_ekf_func, h_measurement_ekf_func, 
                                         Ak_jacobian_ekf_func, Ck_jacobian_ekf_func,
                                         Qk_ekf_assumed, Rk_ekf_assumed, 
                                         x_hat0_ekf_project, P0_ekf_project, 
                                         num_measured_outputs_ekf)
print("EKF instance created.")

### 2.3 NMPC/EMPC Controller Choice and Setup

Let's choose the **Economic MPC (EMPC) from Notebook 5.3** that aims to maximize final product. We'll re-use its CasADi formulation.

In [ ]:
# EMPC Parameters (from Notebook 5.3)
Np_empc_project = 48 
w_P_econ_project = 1.0
R_F_econ_weight_project = 0.01 
S_F_econ_weight_project = 0.1  
F_in_min_empc_project = 0.0; F_in_max_empc_project = 0.05; delta_F_in_max_empc_project = 0.02
S_min_abs_empc_project = 0.05; S_max_abs_empc_project = 20.0; V_max_abs_empc_project = 2.0; Xv_min_abs_empc_project = 0.01
t_batch_final_project = 240 # hr

# --- CasADi EMPC Setup (condensed from 5.3) ---
opti_empc_project = ca.Opti()
X_p_sym = opti_empc_project.variable(nx, Np_empc_project + 1)
U_p_sym = opti_empc_project.variable(nu, Np_empc_project)
x0_p_par = opti_empc_project.parameter(nx)
u_prev_p_par = opti_empc_project.parameter(nu)
params_ode_p = default_params.copy()
obj_p = -w_P_econ_project * (X_p_sym[2, Np_empc_project] * X_p_sym[3, Np_empc_project])
for j in range(Np_empc_project):
    obj_p += R_F_econ_weight_project * (U_p_sym[0, j])**2
    delta_u_e = U_p_sym[0, j] - (u_prev_p_par[0] if j==0 else U_p_sym[0, j-1])
    obj_p += S_F_econ_weight_project * delta_u_e**2
opti_empc_project.minimize(obj_p)

intg_p = ca.integrator('intg_p', 'rk', {'x': x_s_ekf, 'p': u_s_ekf, 'ode': bioreactor_ode_casadi(x_s_ekf, params_ode_p, u_s_ekf)}, 
                       {'tf': Ts_control, 'simplify': True, 'number_of_finite_elements': 4})
opti_empc_project.subject_to(X_p_sym[:,0] == x0_p_par)
for j in range(Np_empc_project):
    res_p = intg_p(x0=X_p_sym[:,j], p=U_p_sym[:,j])
    opti_empc_project.subject_to(X_p_sym[:,j+1] == res_p['xf'])
    opti_empc_project.subject_to(opti_empc_project.bounded(F_in_min_empc_project, U_p_sym[0,j], F_in_max_empc_project))
    delta_u_c_e = U_p_sym[0,j] - (u_prev_p_par[0] if j==0 else U_p_sym[0,j-1])
    opti_empc_project.subject_to(opti_empc_project.bounded(-delta_F_in_max_empc_project, delta_u_c_e, delta_F_in_max_empc_project))
    opti_empc_project.subject_to(X_p_sym[0,j+1] >= Xv_min_abs_empc_project)
    opti_empc_project.subject_to(opti_empc_project.bounded(S_min_abs_empc_project, X_p_sym[1,j+1], S_max_abs_empc_project))
    opti_empc_project.subject_to(X_p_sym[3,j+1] <= V_max_abs_empc_project)

opts_p = {'ipopt.max_iter': 200, 'ipopt.print_level': 0, 'print_time': 0,
            'ipopt.acceptable_tol': 1e-5, 'ipopt.acceptable_obj_change_tol': 1e-5}
opti_empc_project.solver('ipopt', opts_p)
print("EMPC for project formulated.")

## 3. Integrated Closed-Loop Simulation

In [ ]:
sim_time_project = t_batch_final_project
num_sim_steps_project = int(sim_time_project / Ts_control)

x_true_current = x_true_initial.copy()
u_prev_applied = np.array([0.0]) # For EMPC u_prev and EKF predict input

# Initialize EKF with its own initial belief
ekf_project_instance = ExtendedKalmanFilterProject(f_discrete_ekf_func, h_measurement_ekf_func, 
                                                 Ak_jacobian_ekf_func, Ck_jacobian_ekf_func,
                                                 Qk_ekf_assumed, Rk_ekf_assumed, 
                                                 x_hat0_ekf_project, P0_ekf_project, 
                                                 num_measured_outputs_ekf)

# Logging
t_log_proj = np.zeros(num_sim_steps_project + 1)
X_true_log_proj = np.zeros((nx, num_sim_steps_project + 1))
X_hat_log_proj = np.zeros((nx, num_sim_steps_project + 1))
Y_meas_log_proj = np.zeros((num_measured_outputs_ekf, num_sim_steps_project))
U_log_proj = np.zeros((nu, num_sim_steps_project))
P_diag_log_proj = np.zeros((nx, num_sim_steps_project + 1))

X_true_log_proj[:, 0] = x_true_current
X_hat_log_proj[:, 0] = ekf_project_instance.x_hat.flatten()
P_diag_log_proj[:, 0] = np.diag(ekf_project_instance.P)
t_log_proj[0] = 0

U_guess_proj = np.full((nu, Np_empc_project), 0.001)
X_guess_proj = np.tile(ekf_project_instance.x_hat.reshape(nx,1), (1, Np_empc_project + 1))

print(f"Starting Integrated Project Simulation for {num_sim_steps_project} steps...")
for k_proj in range(num_sim_steps_project):
    current_t_proj = k_proj * Ts_control
    print(f"Project Step {k_proj+1}/{num_sim_steps_project} (t={current_t_proj:.1f} hr)", end='\r')

    # 1. Measurement from true plant (S and V)
    y_true_for_meas = np.array([x_true_current[1], x_true_current[3]])
    meas_noise_samp = np.random.multivariate_normal(np.zeros(num_measured_outputs_ekf), Rk_plant_true)
    y_measured_val = y_true_for_meas + meas_noise_samp
    Y_meas_log_proj[:, k_proj] = y_measured_val

    # 2. EKF Step (predict uses u_prev_applied, update uses y_measured and u_prev_applied as D=0 for h)
    x_hat_val, P_val = ekf_project_instance.step(u_prev_applied, y_measured_val.reshape(-1,1), u_prev_applied)
    X_hat_log_proj[:, k_proj] = x_hat_val.flatten()
    P_diag_log_proj[:, k_proj] = np.diag(P_val)

    # 3. EMPC Step (uses estimated state x_hat_val and actual u_prev_applied)
    opti_empc_project.set_value(x0_p_par, x_hat_val)
    opti_empc_project.set_value(u_prev_p_par, u_prev_applied)
    opti_empc_project.set_initial(X_p_sym, X_guess_proj)
    opti_empc_project.set_initial(U_p_sym, U_guess_proj)

    try:
        sol_proj = opti_empc_project.solve()
        U_opt_proj = sol_proj.value(U_p_sym)
        X_pred_proj = sol_proj.value(X_p_sym)
        u_to_apply = U_opt_proj[:, 0]
        X_guess_proj = np.hstack((X_pred_proj[:, 1:], X_pred_proj[:, -1].reshape(nx,1)))
        U_guess_proj = np.hstack((U_opt_proj[:, 1:], U_opt_proj[:, -1].reshape(nu,1)))
    except RuntimeError:
        print(f"\nEMPC Solver failed at project step {k_proj+1}. Holding previous input.")
        u_to_apply = u_prev_applied.flatten()
        # Reset guess
        U_guess_proj = np.full((nu, Np_empc_project), u_prev_applied[0])
        X_guess_proj = np.tile(x_hat_val.reshape(nx,1), (1, Np_empc_project + 1))

    U_log_proj[:, k_proj] = u_to_apply

    # 4. True Plant Evolution (with process noise)
    proc_noise_samp = np.random.multivariate_normal(np.zeros(nx), Qk_plant_true).reshape(nx,1)
    plant_sol_proj_step = solve_ivp(bioreactor_ode_numpy_plant, 
                                    [current_t_proj, current_t_proj + Ts_control], 
                                    x_true_current, 
                                    args=(default_params, u_to_apply[0]), 
                                    dense_output=False, t_eval=[current_t_proj + Ts_control],
                                    method='LSODA')
    x_true_next_no_noise = plant_sol_proj_step.y[:,-1]
    x_true_current = (x_true_next_no_noise.reshape(nx,1) + proc_noise_samp).flatten()
    x_true_current = np.maximum(x_true_current, 0) # Enforce non-negativity

    X_true_log_proj[:, k_proj+1] = x_true_current
    t_log_proj[k_proj+1] = current_t_proj + Ts_control
    u_prev_applied = u_to_apply.reshape(nu,1)
    
    # Check for culture crash
    if x_true_current[0] < 0.001 and x_true_current[1] < 0.001: # Very low Xv and S
        print(f"\nCulture crashed at t={current_t_proj + Ts_control:.1f} hr. Stopping early.")
        # Fill remaining logs
        for i_fill in range(k_proj + 1, num_sim_steps_project):
            U_log_proj[:, i_fill] = u_to_apply
        for i_fill in range(k_proj + 1, num_sim_steps_project + 1):
            X_true_log_proj[:, i_fill] = x_true_current
            X_hat_log_proj[:, i_fill] = x_hat_val.flatten() # last good estimate
            P_diag_log_proj[:, i_fill] = P_val.diagonal()
            t_log_proj[i_fill] = (i_fill) * Ts_control
        if k_proj < num_sim_steps_project:
             Y_meas_log_proj = Y_meas_log_proj[:, :k_proj+1]
        num_sim_steps_project = k_proj + 1 # For plotting
        break

# Final estimate log
X_hat_log_proj[:, num_sim_steps_project] = ekf_project_instance.x_hat.flatten()
P_diag_log_proj[:, num_sim_steps_project] = np.diag(ekf_project_instance.P)

print("\nIntegrated Project Simulation Finished.")

## 4. Analysis and Visualization of Integrated System Performance

In [ ]:
# Adjust time vectors for plotting if simulation stopped early
t_plot = t_log_proj[:num_sim_steps_project+1]
x_true_plot = X_true_log_proj[:, :num_sim_steps_project+1]
x_hat_plot = X_hat_log_proj[:, :num_sim_steps_project+1]
u_plot = U_log_proj[:, :num_sim_steps_project]
y_meas_plot = Y_meas_log_proj[:, :num_sim_steps_project]
p_diag_plot = P_diag_log_proj[:, :num_sim_steps_project+1]
time_u_plot = t_plot[:-1]

fig_proj, axs_proj = plt.subplots(4, 2, figsize=(16, 18), sharex='col')
fig_proj.suptitle('Integrated Output Feedback EMPC with EKF', fontsize=16)

state_labels = ['$X_v$ (g/L)', '$S$ (g/L)', '$P$ (g/L)', '$V$ (L)']
for i in range(nx):
    axs_proj[i,0].plot(t_plot, x_true_plot[i, :], 'b-', label=f'{state_labels[i]} (True)')
    axs_proj[i,0].plot(t_plot, x_hat_plot[i, :], 'r--', label=f'{state_labels[i]} (Est.)')
    if i == 1: # Substrate S
        axs_proj[i,0].plot(time_u_plot, y_meas_plot[0, :], 'gx', ms=4, alpha=0.7, label='$S_{meas}$')
        axs_proj[i,0].axhline(S_min_abs_empc_project, color='m', ls=':', label='$S_{min}$ Cons.')
    if i == 3: # Volume V
        axs_proj[i,0].plot(time_u_plot, y_meas_plot[1, :], 'gx', ms=4, alpha=0.7, label='$V_{meas}$')
        axs_proj[i,0].axhline(V_max_abs_empc_project, color='m', ls=':', label='$V_{max}$ Cons.')
    axs_proj[i,0].set_ylabel(state_labels[i]); axs_proj[i,0].grid(True); axs_proj[i,0].legend()
    
    axs_proj[i,1].plot(t_plot, x_true_plot[i, :] - x_hat_plot[i, :], 'm-', label=f'Error {state_labels[i]}')
    axs_proj[i,1].plot(t_plot, 2*np.sqrt(p_diag_plot[i,:]), 'k:', label='$\pm 2\sigma$')
    axs_proj[i,1].plot(t_plot, -2*np.sqrt(p_diag_plot[i,:]), 'k:')
    axs_proj[i,1].set_ylabel(f'Error {state_labels[i]}'); axs_proj[i,1].grid(True); axs_proj[i,1].legend()

axs_proj[nx-1,0].set_xlabel('Time (hr)'); axs_proj[nx-1,1].set_xlabel('Time (hr)')

fig_ctrl, ax_ctrl = plt.subplots(1,1,figsize=(12,5))
ax_ctrl.step(time_u_plot, u_plot[0,:], 'k-', where='post', label='$F_{in}$ (L/hr)')
ax_ctrl.axhline(F_in_max_empc_project, color='m', ls='--', label='$F_{in,max}$ Cons.')
ax_ctrl.set_ylabel('Feed Rate $F_{in}$'); ax_ctrl.set_xlabel('Time (hr)')
ax_ctrl.grid(True); ax_ctrl.legend(); ax_ctrl.set_title('Control Input Trajectory')

plt.tight_layout(rect=[0, 0, 1, 0.96])
plt.show()

final_total_product_project = x_true_plot[2, -1] * x_true_plot[3, -1]
print(f"Final true conditions at t={t_plot[-1]:.1f} hr (EMPC with EKF):")
print(f"  Xv = {x_true_plot[0, -1]:.3f} g/L, Est. Xv = {x_hat_plot[0,-1]:.3f}")
print(f"  S  = {x_true_plot[1, -1]:.3f} g/L, Est. S  = {x_hat_plot[1,-1]:.3f}")
print(f"  P  = {x_true_plot[2, -1]:.3f} g/L, Est. P  = {x_hat_plot[2,-1]:.3f}")
print(f"  V  = {x_true_plot[3, -1]:.3f} L, Est. V  = {x_hat_plot[3,-1]:.3f}")
print(f"  Total True Product = {final_total_product_project:.3f} g")

## 5. Discussion and Further Exploration

*   **Impact of Estimation:** How does the performance of the EMPC (in terms of final product and constraint handling) compare to the idealized full-state feedback case (Notebook 5.3)? Are there significant differences?
*   **EKF Tuning:** The choice of $Q_K$ (process noise covariance for EKF) and $R_K$ (measurement noise covariance for EKF) is critical. 
    *   If the EKF's $Q_K$ is too small, it might trust its model too much and be slow to react to true state deviations not captured by its model, potentially leading to poor MPC decisions.
    *   If $R_K$ is too small, it might overfit to noisy measurements, leading to noisy state estimates and potentially chattering control by the MPC.
    *   **Exercise:** Try different $Q_K$ and $R_K$ values for the EKF (in `Qk_ekf_assumed`, `Rk_ekf_assumed`) and observe the impact on both estimation accuracy and overall EMPC performance. How robust is the system if the EKF's noise assumptions don't perfectly match the true plant noise (`Qk_plant_true`, `Rk_plant_true`)?
*   **MPC Robustness to Estimation Errors:** How do errors in state estimation propagate through the MPC's predictions and affect its decisions? Is the MPC inherently robust to some level of estimation error, or does it require very accurate estimates?
*   **Infrequent Measurements:** This simulation assumed S and V are measured at every `Ts_control`. Modify the EKF update logic to only use these measurements when they would realistically be available (e.g., V continuously, S perhaps every few hours). How does this affect estimation and control? (This would require a more event-driven EKF update).
*   **Model Mismatch:** What if the `default_params` used by the EKF and MPC are different from slightly perturbed `true_plant_params` used in `bioreactor_ode_numpy_plant`? This would introduce structural model error on top of noise.

This mini-project demonstrates the complexities and interdependencies in a realistic output feedback MPC system. Both the estimator and the controller need to be well-designed and tuned for good overall performance.

## 6. Key Takeaways

*   Combining NMPC/EMPC with a state estimator like the EKF enables control based on noisy and partial measurements, which is crucial for real-world bioreactor applications.
*   The performance of the overall system depends on the quality of both the state estimator and the NMPC/EMPC controller design and tuning.
*   Simulating the integrated system with realistic noise models helps to understand potential performance limitations and the sensitivity to various tuning parameters.
*   Cascaded effects are important: poor estimation will lead to poor control, even if the controller itself is well-designed for known states.

This notebook serves as a capstone for our bioreactor control case studies using first-principles models. The next part of the series (**Part 6: Data-Driven Modeling for MPC**) will explore scenarios where such detailed mechanistic models might not be available or are difficult to develop, and how data-driven approaches can fill that gap.